In [9]:
import math
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from problem_data import number_of_cycles, points_by_cycle

In [11]:
number_of_points = 10

In [43]:
def distance (source, target):
    return np.linalg.norm(target - source, ord=2)

def make_distance_dict (points, number_of_points):
    assert number_of_points <= points.shape[0]
    return {
        (i, j): distance(points[i], points[j])
        for i in range(number_of_points)
        for j in range(i)
    }

def make_variables (model, cycle_number, indices, costs):
    variables = model.addVars(indices, obj=costs, vtype=GRB.BINARY, name=f'e{cycle_number}')
    for i, j in variables.keys():
        variables[j, i] = variables[i, j]  # edge in opposite direction
    return variables

def make_variables_by_cycle (model, distances_by_cycle):
    variables_by_cycle = [
        make_variables(
            model,
            cycle_number,
            distances_by_cycle[cycle_number].keys(),
            distances_by_cycle[cycle_number]
        )
        for cycle_number in range(number_of_cycles)
    ]
    return variables_by_cycle

def make_constraints_by_cycle (model, variables_by_cycle, number_of_cycles, number_of_points):
    constraints = [
        [
            model.addConstrs(
                variables_by_cycle[cycle_number].sum(point_index, '*') == 2
                for point_index in range(number_of_points)
            )
        ]
        for cycle_number in range(number_of_cycles)
    ]
    return constraints

In [44]:
distances_by_cycle = [
    make_distance_dict(points_by_cycle[cycle_number], number_of_points)
    for cycle_number in range(number_of_cycles)
]

In [47]:
# mathematical model

model = gp.Model()

## variables

variables_by_cycle = make_variables_by_cycle(model, distances_by_cycle)

## constraints

constraints_by_cycle = make_constraints_by_cycle(model, variables_by_cycle, number_of_cycles, number_of_points)
